In [ ]:
# importing requests and json
import torch
import torchvision
import requests, json
from enum import Enum
import keras
import tensorflow as tf
!pip install anvil-uplink
!pip install git+https://github.com/ricardodeazambuja/colab_utils.git
!pip install google-cloud-speech
!pip install pydub
from google.cloud import speech
import anvil.server
import numpy as np
!pip install psycopg2
import psycopg2
!pip install timm
import timm
from PIL import Image
!pip install -U keras-efficientnet-v2
!pip install -U keras_cv_attention_models 
from colab_utils import getAudio
from scipy.io.wavfile import write
import io
from io import BytesIO
!pip install colorthief
from colorthief import ColorThief
from google.colab import files
import os
!pip install webcolors
from scipy.spatial import KDTree
from webcolors import CSS3_HEX_TO_NAMES, hex_to_rgb
!pip install openai
import openai
import random




In [ ]:
# some initial setup

is_woman = True # changed in code depending on who is the user
preferred_skin_tone = "asian"

anvil.server.connect("anvil server key")

openai.api_key = "" #api key ere

class Temperature(Enum):
    FREEZING = 1
    MODERATE = 2
    HOT = 3
    
temperature_mapping = {
  "sandal" : 3,           # probs not gonna use
  "runningshoe" : 2,           # probs not gonna use
  "kimono" : 3 ,
  "overskirt" : 3,
  "labcoat" : 2,
  "militaryuniform" : 2,
  "brassiere" : 3,           # probs not gonna use
  "poncho" : 2,
  "apron":3,
  "miniskirt":3,
  "jersey" : 3,
  "cardigan" : 3,
  "sweatshirt" : 2,
  "furcoat" :1,
  "cloak" : 2,
  "suit" : 2,
  "jean":3,
  "bowtie" : 3,           # probs not gonna use
  "trenchcoat" : 2,
  "necklace" : 3,           # probs not gonna use
  "purse" : 3,           # probs not gonna use
  "sock" : 2,           # probs not gonna use
  "bikini" : 3,           # probs not gonna use
  "backpack" : 2,           # probs not gonna use
  "mask" : 1,           # probs not gonna use
}

# tops_list = ["kimono", "labcoat", "militaryuniform", "poncho", "apron", "jersey", "cardigan", "sweatshirt", "furcoat", "cloak", "suit", "trenchcoat"]
# bottoms_list = ["overskirt", "miniskirt", "jean"]
tops_list = ["kimono", "labcoat", "militaryuniform", "poncho", "apron", "jersey", "cardigan", "sweatshirt", "furcoat", "cloak", "suit", "trenchcoat", "shirt", "sweater", 'jacket', 'tshirt', 'tanktop']
bottoms_list = ["overskirt", "miniskirt", "jean", "sweatpants", "shorts", 'jeans']

'''GETTING THE WEATHER'''
# base URL
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
# City Name 
CITY = "Montreal"
# API key 
API_KEY = "69f2add663081b7872b6c7fbb3a22d3e"
#https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}
# upadting the URL
URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY
# HTTP request
response = requests.get(URL)
# checking the status code of the request
if response.status_code == 200:
   # getting data in the json format
   data = response.json()
   # getting the main dict block
   main = data['main']
   # getting temperature
   temperature = round(main['temp'] - 273, 2)
   # getting the humidity
   humidity = main['humidity']
   # getting the pressure
   pressure = main['pressure']
   # weather report
   report = data['weather']
   print(f"{CITY:-^30}")
   print(f"Temperature: {temperature}")
   print(f"Humidity: {humidity}")
   print(f"Pressure: {pressure}")
   print(f"Weather Report: {report[0]['description']}")

# sorting based on temperature 

if -50 <= temperature < 5:
  current_temp = Temperature.FREEZING
elif 5 <= temperature <= 20:
  current_temp = Temperature.MODERATE
else: # temps > 20
  current_temp = Temperature.HOT
print(current_temp)

freezing_storage = {}
moderate_storage = {}
hot_storage = {}

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/speech-to-text-key.json'
os.environ['PROJECT_ID'] = 'codejam-12'
!ls -l $GOOGLE_APPLICATION_CREDENTIALS


-50 to 5: freezing : coats, fur coats, boots

5 to 20: moderate : jackets, cardigans, sweatshirts, shoes

20+ hot : tshirts, swimsuits, sandals

use 1k for clothing thickness, 21k is too precise

In [ ]:
@anvil.server.callable
def change_skintone(tone):
  preferred_skin_tone = tone

@anvil.server.callable
def change_style(style):
  is_woman = style



In [ ]:
from pydub import AudioSegment
# setup audio recon

triggers = {"tailor", "taylor", "tayler", "tailer", "taler", "teller", "tyler", "toiler", "cater", "peter", "meter", "beater", "Baylor", "Paylor", "Taylor's", "Tailor's", "sailor", "hayler", "hailer", "trainer"}
command_bank = {"drip", "trip", "grip", "outfit", "outfits", "suggest", "wear", "weather", "clothes", "fits", "fit", "dress", "gym", "frigid", "icy", "cool"}
cold_terms = {"cold", "snow", "frosty", "freeze", "winter", "freezing", "chilly", "icy"}
moderate_terms = {"spring", "fall", "autumn", "moderate", "temperate", "soft", }
hot_terms = {"inside", "hot", "sunny", "summer", "sweating", "home", "canicule", "heatwave", "heat", "overheating", "tropical", "warm", "muggy", "boiling", "scorching", "sweltering", "searing", "blistering"}

# returns a url to the image of the person wearing clothes
@anvil.server.callable
def get_command(wav_bytes):
  # audio, sr = getAudio() #html button to record audio
  # print(len(audio))

  # wav_bytes = io.BytesIO()
  # write(wav_bytes, 44100, audio)

  # 
  audio_b = wav_bytes.get_bytes()
  audio_ogg = io.BytesIO(audio_b)
  audio_wav = io.BytesIO()

  file = AudioSegment.from_file(audio_ogg)
  file.export(audio_wav, format='wav', parameters=["-ac", "1"], bitrate="192k")
  audio_wav.seek(0)
  audio = speech.RecognitionAudio(content=audio_wav.read())
  config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz= file.frame_rate, # Value of the sample rate of the dell mics, TODO: Parse wav for proper rate
        language_code="en-US",
    )
  
  speech_client = speech.SpeechClient()
  response = speech_client.recognize(config=config, audio=audio)

  words = ""

  for result in response.results:
    #print("Transcript: {}".format(result.alternatives[0].transcript))
    words += result.alternatives[0].transcript
    print("words: ", words)
  output_words = words
  tailor_detected = False
  command_detected = False
  weather = current_temp
  return_string = "" 
  for word in words.split():
    print("checking for 'tailor':", word)
    if word.lower() in triggers:
      tailor_detected = True
      return_string += "trigger detected: "
      break

  if tailor_detected:
    for word in words.split():
      print("checking word: ", word)
      if word.lower() in command_bank:
        return_string += "command detected"
        command_detected = True
      elif word.lower() in cold_terms:
        return_string += "cold weather term: " + word + " :"
        weather = Temperature.FREEZING
        command_detected = True
      elif word.lower() in moderate_terms:
        return_string += "moderate weather term: " + word + " :"
        weather = Temperature.MODERATE
        command_detected = True
      elif word.lower() in hot_terms:
        return_string += "hot weather term: " + word + " :"
        weather = Temperature.HOT
        command_detected = True

  else:
    return_string += "trigger word not detected in " + output_words + " (" + str(len(audio_b)) + ") at " + str(file.frame_rate) +", try again"

  if command_detected:
    if is_woman:
      gender = "woman"
    else:
      gender = "man"
    print("Generating...")
    url = generate_image(gender, preferred_skin_tone, weather)
  else: 
    return return_string, None # No commands, dont bother
    # sends the url as string to the client
    # anvil.server.call("set_image", url)

    #do smth with the url
  return_string += output_words + " (" + str(len(audio_b)) + ") at " + str(file.frame_rate)
  return return_string, url

  # should return success, weather (0 = weather api, 1 = cold, 2 = moderate, 3 = hot)


In [ ]:
@anvil.server.callable

def generate_image(gender, skintone, weather = current_temp):
  prompt = "modern photo of {} {} wearing a {} {} and {} {}, white background"
  #prompt = "realistic photo of {} {} wearing a {} {} and {} {}"

  if weather == Temperature.FREEZING:
    clothing_top, color_top, clothing_bottom, color_bottom = select_clothes(freezing_storage, moderate_storage)
  
  elif weather == Temperature.MODERATE:
    if random.randint(0,1) == 0:
      clothing_dict = freezing_storage
    else:
      clothing_dict = hot_storage
    clothing_top, color_top, clothing_bottom, color_bottom = select_clothes(moderate_storage, clothing_dict)

  elif weather == Temperature.HOT:
    clothing_top, color_top, clothing_bottom, color_bottom = select_clothes(moderate_storage, hot_storage)

  final_prompt = prompt.format(skintone, gender, color_top, clothing_top, color_bottom, clothing_bottom)
  print(final_prompt)

  response = openai.Image.create(
  prompt=final_prompt,
  n=1,
  size="512x512"
  )
  image_url = response['data'][0]['url']

  print(clothing_top, color_top, clothing_bottom, color_bottom)
  print(image_url)

  return image_url

#test function
#generate_image("man", "asian", Temperature.MODERATE)

In [ ]:
generate_image("man", "caucasian", Temperature.FREEZING)

In [ ]:
def select_clothes(dict1, dict2):
  if random.randint(0,1) == 0:
    clothing_dict = dict1
  else:
    clothing_dict = dict2
  
  #pick top
  # while True:
  #   clothing_top, color_list_top = random.choice(list(clothing_dict.items()))
  #   if clothing_top in tops_list:
  #     break;
  clothing_top = random.choice(list(set(clothing_dict) & set(tops_list)))
  color_top = random.choice(clothing_dict[clothing_top])

  #pick bottom
  # while True:
  #   clothing_bottom, color_list_bottom = random.choice(list(clothing_dict.items()))
  #   if clothing_bottom in bottoms_list:
  #     break;
  clothing_bottom = random.choice(list(set(clothing_dict) & set(bottoms_list)))
  color_bottom = random.choice(clothing_dict[clothing_bottom])

  return clothing_top, color_top, clothing_bottom, color_bottom

# clothing_top, color_top, clothing_bottom, color_bottom = select_clothes(freezing_storage, moderate_storage)
# print(clothing_top, color_top, clothing_bottom, color_bottom)

In [ ]:
@anvil.server.callable
def get_weather():
  #print(f"{CITY:-^30}\n Temperature: {temperature}\n Weather Report: {report[0]['description']}")
  return f"{CITY:-^30}\n Temperature: {temperature}°C\n Humidity: {humidity}%\n Weather: {report[0]['description']}"



In [ ]:
get_weather()

In [ ]:
# color converter

def convert_rgb_to_names(rgb_tuple):
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

def get_color(image):

  width,height = image.size
  new_width = width*2//5
  new_height = height*2//5
  left = (width - new_width)/2
  right = (width + new_width)/2
  top = (height - new_height)/2
  bottom = (height + new_height)/2

  # Crop the center of the image
  im = image.crop((left, top, right, bottom))
  im.save("image_cropped.png")

  color_thief = ColorThief('image_cropped.png')
  dominant_color = color_thief.get_color(quality=1)
  !rm image_cropped.png
  from webcolors import rgb_to_name
  named_color = convert_rgb_to_names(dominant_color)
  return(named_color)


In [ ]:
image5 = Image.open('KDF-codejam12/Untitled.png')
get_color(image5)

'firebrick'

In [ ]:
!git clone https://github.com/iveykun/KDF-codejam12.git

Cloning into 'KDF-codejam12'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 81 (delta 32), reused 40 (delta 9), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [ ]:
# adding clothing by default
whitecolor = 'cyan'
blackcolor = 'black'
freezing_storage['furcoat'] = [blackcolor, "grey"]
freezing_storage['jeans'] = [blackcolor, "blue", "grey"]
freezing_storage['jacket'] = [blackcolor, "blue", "grey", "green"]

moderate_storage['cardigan'] = [whitecolor, blackcolor, "blue", "green", "grey"]
moderate_storage['shorts'] = [whitecolor, blackcolor, "blue", "grey"]
moderate_storage['shirt'] = [whitecolor, blackcolor, "blue", "green", "grey"]
moderate_storage['sweatpants'] = [whitecolor, blackcolor, "grey"]
moderate_storage['sweater'] = [whitecolor, blackcolor, "blue", "green", "grey"]
moderate_storage['jeans'] = [blackcolor, "blue", "grey"]

hot_storage['jersey'] = [whitecolor, blackcolor, "blue", "green", "grey"]
hot_storage['shorts'] = [whitecolor, blackcolor, "blue", "grey",]
hot_storage['miniskirt'] = [whitecolor, blackcolor, "blue", "green", "grey"]
hot_storage['tshirt'] = [whitecolor, blackcolor, "blue", "green", "grey"]
hot_storage['sweatpants'] = [whitecolor, blackcolor, "blue", "green", "grey"]
hot_storage['tanktop'] = [whitecolor, blackcolor, "blue", "green", "grey"]


In [ ]:
@anvil.server.callable
def imagenet1kmedia(mediaobject):
  #response = requests.get(str(link))
  obj = mediaobject.get_bytes()
  img = Image.open(BytesIO(obj))
  #model = timm.create_model('mobilenetv3_large_100_miil_in21k', pretrained=True)
  model = torchvision.models.resnet101(pretrained=True)

  #img_1 = Image.open("drive/My Drive/codejam/test1.png").convert('RGB')
  #img_1 = Image.open(image).convert('RGB')

  preprocess = torchvision.transforms.Compose([
          torchvision.transforms.Resize(256),
          torchvision.transforms.CenterCrop(224),
          torchvision.transforms.ToTensor(),
          torchvision.transforms.Normalize(
          mean=[0.485, 0.456, 0.406],
          std=[0.229, 0.224, 0.225]
      )])
  # CHANGE THIS FOR DIFFERENT IMAGES 
  img_preprocessed = preprocess(img)
  # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  batch_img_cat_tensor = torch.unsqueeze(img_preprocessed, 0)
  model.eval()
  # source: https://vitalflux.com/pytorch-load-predict-pretrained-resnet-model/
  out = model(batch_img_cat_tensor)
  
  with open('KDF-codejam12/imagenet_classes.txt') as f:
      labels = [line.strip() for line in f.readlines()]
  _, index = torch.max(out, 1)
  percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
  print(labels[index[0]], percentage[index[0]].item())
  _, indices = torch.sort(out, descending=True)
  [(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]
  tag = labels[index[0]].split(",")[0].replace(" ", "")
  # so add imagebytes, temp, tag
  newcolor = get_color(img)
  #print(newcolor)
  temp = temperature_mapping[tag]

  if temp == 1:
    if tag not in freezing_storage:
      freezing_storage[tag] = [newcolor]
      moderate_storage[tag] = [newcolor]
    else:
      freezing_storage[tag].append(newcolor)
      moderate_storage[tag].append(newcolor)
  elif temp == 2:
    if tag not in moderate_storage:
      freezing_storage[tag] = [newcolor]
      moderate_storage[tag] = [newcolor]
      hot_storage[tag] = [newcolor]
    else:
      freezing_storage[tag].append(newcolor)
      moderate_storage[tag].append(newcolor)
      hot_storage[tag].append(newcolor)
  else: # must be 3
    if tag not in hot_storage:
      moderate_storage[tag] = [newcolor]
      hot_storage[tag] = [newcolor]
    else:
      moderate_storage[tag].append(newcolor)
      hot_storage[tag].append(newcolor)


  return (newcolor, labels[index[0]].split(",")[0].replace(" ", ""), percentage[index[0]].item())
  _, indices = torch.sort(out, descending=True)
  [(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]



In [ ]:
'''imagenet 21 000'''
@anvil.server.callable
def imagenet21k():
  # is more accurate but it's TOO detailed
  import keras_efficientnet_v2
  from keras_cv_attention_models import imagenet
  import numpy
  model = keras_efficientnet_v2.EfficientNetV2B3(pretrained='imagenet21k', num_classes=21843)
  img_1 = Image.open("drive/My Drive/codejam/test1.png").convert('RGB')

  # there's gotta be a better way to do this, but converting PIL to numpy
  na = numpy.array(img_1)
  imm = tf.image.resize(na, model.input_shape[1:-1]) 
  imm2 = tf.keras.applications.imagenet_utils.preprocess_input(tf.expand_dims(imm, 0), mode='tf')
  pred = model(imm2).numpy()
  print(imagenet.decode_predictions_imagenet21k(pred))
  return(imagenet.decode_predictions_imagenet21k(pred))
  #print(keras.applications.imagenet_utils.decode_predictions(pred)[0])

In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored